# VERY INCOMPLETE

In [27]:
import numpy as np

import pandas as pd

from ast import literal_eval

import nltk

Load `chaps_df` and undo array-to-list.

In [7]:
chaps_df = pd.read_csv('../data/animorphs_chaps.csv')

for col in ['clean','clean_no_stops', 'vec_clean']:
    chaps_df[col] = chaps_df[col].map(literal_eval) # as lists
    
for col in ['vec_clean']:
    chaps_df[col] = chaps_df[col].map(np.array) # as array
    
chaps_df.head()

,book,chapter,text,authenticity,clean,clean_no_stops,vec_clean
0,1,1,"My name is Jake. That's my first name, obvious...",1,"[name, thats, first, name, obviously, cant, te...","[my, name, is, jake, thats, my, first, name, o...","[0.050813198, 0.21546867, -0.11217794, 0.05301..."
1,1,2,"""A flying saucer?"" Marco said. He did laugh. T...",1,"[flying, saucer, laugh, looked, could, feel, h...","[a, flying, saucer, marco, said, he, did, laug...","[0.0059409393, 0.23846294, -0.09167185, 0.0067..."
2,1,3,<They have come to destroy you.>\nIt was stran...,1,"[come, destroy, strange, way, knew, telling, t...","[they, have, come, to, destroy, you, it, was, ...","[0.00516426, 0.25513598, -0.040639274, -0.0519..."
3,1,4,<Yeerks!>\nThe twin red lights slowed. They tu...,1,"[yeerks, twin, red, lights, slowed, turned, ci...","[yeerks, the, twin, red, lights, slowed, they,...","[-0.027513022, 0.24088845, 0.00715725, -0.0299..."
4,1,5,"The Hork-Bajir pointed his gun, or whatever it...",1,"[hork, bajir, pointed, gun, whatever, around, ...","[the, hork, bajir, pointed, his, gun, or, what...","[-0.037028935, 0.29484826, 0.030976577, -0.076..."


Load `books_df` and undo array-to-list.

In [8]:
books_df = pd.read_csv('../data/animorphs_books.csv')

for col in ['clean','book_vec']:
    books_df[col] = books_df[col].map(literal_eval) # as lists
    
for col in ['book_vec']:
    books_df[col] = books_df[col].map(np.array) # as array
    
books_df.head()

,book,text,authenticity,clean,book_vec
0,1,"My name is Jake. That's my first name, obvious...",1,"[name, thats, first, name, obviously, cant, te...","[0.0010895184, 0.2081184, -0.05931624, 0.01286..."
1,2,My name is Rachel. I won't tell you my last na...,1,"[name, tell, last, name, none, us, ever, tell,...","[-0.00226549, 0.22271992, -0.065148115, 0.0113..."
2,3,My name is Tobias. A freak of nature. One of a...,1,"[name, freak, nature, kind, tell, last, name, ...","[-0.0129256295, 0.21558568, -0.05230921, 0.003..."
3,4,My name is Cassie.\nI can't tell you my last n...,1,"[name, cant, tell, last, name, wish, could, ca...","[-0.0025382496, 0.21817772, -0.069260634, 0.00..."
4,5,My name is Marco.\nI can't tell you my last na...,1,"[name, cant, tell, last, name, live, believe, ...","[-0.00024276103, 0.22412194, -0.05454963, 0.00..."


After consulting [this post](https://programminghistorian.org/en/lessons/introduction-to-stylometry-with-python) about historical methods of stylometry, we have decided to implement John Burrow's Delta scoring to compare its appropriateness to that of our W2V+SVM model. We avoided copying code directly, but much of this methodology directly follows from the post.

It may be that we need to differentiate between authors within the group of ghostwriters. For this analysis, though, we will treat them as one, as we would like a model that can operate without that level of information.

In [29]:
def delta_one_out(book_out, n, books_df=books_df):

    train_inds = books_df['book'] != book_out
    test_inds = books_df['book'] == book_out
    
    X_b = books_df['clean'].loc[train_inds]
    y_b = books_df['authenticity'].loc[train_inds]
    
    X_b_test = books_df['clean'].loc[test_inds]
    
    return burrows_delta(X_b, y_b, X_b_test, n)

In [ ]:
# adapted from
# programminghistorian.org/en/lessons/introduction-to-stylometry-with-python
def burrows_delta(X, y, X_test, n):
    
    # format subcorpuses
    authors = {a:{'corpus':[], 'delta':0} for a in y.values}
    authors['whole'] = {'corpus':[]} # whole corpus
    authors['test'] = {'corpus':[]}

    for book, auth in zip(X.values, y.values):
        authors['whole']['corpus'] += book
        authors[auth]['corpus'] += book
    authors['test']['corpus'] = list(np.squeeze(X_test.values))
            
    # n most common word frequncies across whole corpus
    authors['whole']['freqs'] = {f[0]:{'freq':f[1]/len(
        authors['whole']['corpus'])} for f in nltk.FreqDist(
        authors['whole']['corpus']).most_common(n)}
    
    for auth in authors.keys():
        authors[auth]['freqs'] = {}

In [68]:
# # adapted from
# # programminghistorian.org/en/lessons/introduction-to-stylometry-with-python
# def burrows_delta_BINARY(X, y, X_test, n):
    
#     # format subcorpuses
#     whole = []
#     auth_corpus = []
#     gw_corpus = []
#     for book, auth in zip(X.values, y.values):
#         whole += book
#         if auth == 1:
#             auth_corpus += book
#         elif auth == 0:
#             gw_corpus += book
            
#     test_corpus = list(np.squeeze(X_test.values))
            
#     # n most common word frequncies across whole corpus
#     whole_freqs = {f[0]:{'freq':f[1]/len(whole)}
#                    for f in nltk.FreqDist(whole).most_common(n)}
    
#     auth_freqs = {}
#     gw_freqs = {}
#     test_freqs = {}
#     deltas = {'auth':0,'gw':0}
#     for word in whole_freqs.keys():
        
#         # frequencies across subcorpuses
#         auth_freqs[word] = {'freq':
#                             auth_corpus.count(word)/len(auth_corpus)}
#         gw_freqs[word] = {'freq':
#                           gw_corpus.count(word)/len(gw_corpus)}
#         test_freqs[word] = {'freq':
#                             test_corpus.count(word)/len(test_corpus)}
        
#         # overall avg and std
#         whole_freqs[word]['mean'] = \
#             (auth_freqs[word]['freq'] + gw_freqs[word]['freq'])/2
#         whole_freqs[word]['std'] = np.sqrt(
#             (auth_freqs[word]['freq'])**2 + (gw_freqs[word]['freq'])**2)
        
#         # z-scores
#         auth_freqs[word]['z'] = \
#             ((auth_freqs[word]['freq'] - whole_freqs[word]['mean']) 
#              / whole_freqs[word]['std'])
#         gw_freqs[word]['z'] = \
#             ((gw_freqs[word]['freq'] - whole_freqs[word]['mean']) 
#              / whole_freqs[word]['std'])
#         test_freqs[word]['z'] = \
#             ((test_freqs[word]['freq'] - whole_freqs[word]['mean'])
#              / whole_freqs[word]['std'])
        
#         # delta subscores
#         deltas['auth'] += abs(test_freqs[word]['z'] - auth_freqs[word]['z'])
#         deltas['gw'] += abs(test_freqs[word]['z'] - gw_freqs[word]['z'])
        
#     # deltas
#     deltas = {k:v/len(whole_freqs.keys()) for k,v in deltas.items()}
#     return deltas

In [89]:
def delta_auth(b, n=100):
    deltas = delta_one_out(b, n)
    return int(deltas['auth'] < deltas['gw'])

In [77]:
int(temp['auth'] < temp['gw'])

(0, {'auth': 0.35452751839622726, 'gw': 0.22714458803448942})

In [81]:
results = pd.read_csv('../data/results_w2vsvm.csv')[
    ['book','book pred proba','authenticity']]

,book,book pred proba,authenticity
0,1,0.969500,1
1,2,0.983391,1
2,3,0.980554,1
3,4,0.981422,1
4,5,0.985339,1


In [92]:
results['book'].map(delta_auth)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     0
8     1
9     1
10    1
11    1
12    0
13    0
14    1
15    0
16    0
17    0
18    0
19    0
20    1
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
Name: book, dtype: int64